In [61]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import os


In [73]:
data_path="covid_data/"
normal_path= data_path + '0/'
covid_path = data_path + '1/'

train_filenames = [] # all image name
train_labels = [] # all image labels

# label o for normal
for filename in os.listdir(normal_path):
    train_filenames.append(normal_path + filename)
    train_labels.append(0)

# label 1 for covid
for filename in os.listdir(covid_path):
    train_filenames.append(covid_path + filename)
    train_labels.append(1)


# Random data splitting
X_train, X_valid, y_train, y_valid = train_test_split(train_filenames, train_labels, test_size=0.2)

# covert to tensorflow constant
train_filenames = tf.constant(X_train)
valid_filenames = tf.constant(X_valid)
train_labels = tf.constant(y_train)
valid_labels = tf.constant(y_valid)


In [74]:
# this function help to decode and resize the image to resized img and its label
def decode_and_resize(filename, label):
    image_string = tf.io.read_file(filename)            
    image_decoded = tf.image.decode_jpeg(image_string) 
    image_resized = tf.image.resize(image_decoded, [256, 256]) / 255.0 # reset size，and normalized
    return image_resized, label


In [75]:
# training dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_filenames, train_labels))
train_dataset = train_dataset.map(map_func=decode_and_resize, # call decode_and_resize method and covert file to features and labels
    num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_dataset = train_dataset.batch(32) # amount of data per batch

# valid_dataset
valid_dataset = tf.data.Dataset.from_tensor_slices((valid_filenames, valid_labels))
valid_dataset = valid_dataset.map(
    map_func=decode_and_resize,
    num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.batch(32)


In [76]:
model = tf.keras.Sequential([
    # Convolutional Layer，32 filter，3*3，strides is 1
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(256, 256, 3)),
    # Pooling Layer，2*2，trides are 2
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 5, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)


In [77]:
model.summary()


Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 123, 123, 32)      25632     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 61, 61, 32)        0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 119072)            0         
_________________________________________________________________
dense_22 (Dense)             (None, 64)                7620672   
_________________________________________________________________
dense_23 (Dense)             (None, 2)               

In [78]:
test_dataset = tf.data.Dataset.from_tensor_slices((valid_filenames, valid_labels))
test_dataset = test_dataset.map(_decode_and_resize)
test_dataset = test_dataset.batch(32)

print(model.evaluate(test_dataset))


20/20 [==============================] - 7s 338ms/step - loss: 0.6942 - sparse_categorical_accuracy: 0.4510
[0.6959580183029175, 0.4248788356781006]


In [45]:

model.metrics_names



['loss', 'sparse_categorical_accuracy']